# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 58 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.06161


extracting tarball to tmp_2212.06161...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.06168


extracting tarball to tmp_2212.06168... done.
Retrieving document from  https://arxiv.org/e-print/2212.06174


...

/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.06666/natbib.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2212.06683... done.
Retrieving document from  https://arxiv.org/e-print/2212.06791


extracting tarball to tmp_2212.06791...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.06791/paper1.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'TablaHerbig' from 'tmp_2212.06791/TablaHerbig.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'Tabla3' from 'tmp_2212.06791/Tabla3.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_dec

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06434-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06434) | **PHANGS--JWST First Results: ISM structure on the turbulent Jeans scale  in four disk galaxies observed by JWST and ALMA**  |
|| Sharon E. Meidt, et al. -- incl., <mark>Eva Schinnerer</mark>, <mark>Janice C. Lee</mark>, <mark>Frank Bigiel</mark>, <mark>Jonathan D. Henshaw</mark>, <mark>Daizhong Liu</mark>, <mark>Rowan J. Smith</mark> |
|*Appeared on*| *2022-12-14*|
|*Comments*| *15 pages, 4 figures. Accepted as part of a PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| JWST/MIRI imaging of the nearby galaxies IC 5332, NGC 628, NGC 1365 and NGC 7496 from PHANGS reveals a richness of gas structures that in each case form a quasi-regular network of interconnected filaments, shells and voids. We examine whether this multi-scale network of structure is consistent with the fragmentation of the gas disk through gravitational instability. We use FilFinder to detect the web of filamentary features in each galaxy and determine their characteristic radial and azimuthal spacings. These spacings are then compared to estimates of the most Toomre-unstable length (a few kpc), the turbulent Jeans length (a few hundred pc) and the disk scale height (tens of pc) reconstructed using PHANGS-ALMA observations of the molecular gas as a dynamical tracer. Our analysis of the four galaxies targeted in this work indicates that Jeans-scale structure is pervasive. Future work will be essential for determining how the structure observed in gas disks impacts not only the rate and location of star formation but also how stellar feedback interacts positively or negatively with the surrounding multi-phase gas reservoir. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06598-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06598) | **Synthetic dust polarization emission maps at 353 GHz for an observer  placed inside a Local Bubble-like cavity**  |
|| E. Maconi, et al. -- incl., <mark>P. Hennebelle</mark>, <mark>R. J. Smith</mark> |
|*Appeared on*| *2022-12-14*|
|*Comments*| **|
|**Abstract**| We present a study of synthetic observations of polarized dust emission at 353 GHz as seen by an observer within a cavity. The cavity's physical properties are comparable to the Local Bubble in which the Solar System is embedded. We find that the combination of the local density enhancement and the coherent magnetic field in the walls of the cavity makes the Local Bubble a translucent polarization filter to the emission coming from beyond its domains. This underlines the importance of studying the Local Bubble in detail, as it is a foreground screen to Galactic astronomy and cosmology. We find that the magnetic field lines inferred from synthetic dust polarization data are qualitatively in agreement, at low and middle latitudes (|b|<60 deg), with the all-sky maps of polarized emission at 353 GHz by the Planck satellite. This is in agreement with current models of the magnetic field structure in the Local Bubble walls. However, we also find that the polarized signal coming from the selected cavity is insufficient to explain the regular polarization orientation patterns observed by Planck toward the polar Galactic caps. This may be due to the advanced stage of evolution of the candidate bubble and to the limited simulation domain. Finally, we show that from our synthetic polarization maps, it is difficult to distinguish between an open and a closed Galactic cap using the inferred magnetic field morphology alone. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06161-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06161) | **Common envelope evolution and triple dynamics as potential pathways to  form the inner white dwarf + brown dwarf binary of the triple star system  Gaia 0007-1605**  |
|| F. Lagos, M. Zorotovic, <mark>M. R. Schreiber</mark>, B. T. Gänsicke |
|*Appeared on*| *2022-12-14*|
|*Comments*| *7 pages, 4 figures; accepted for publication in MNRAS*|
|**Abstract**| The recently discovered system Gaia 0007-1605 consisting of a white dwarf with a close brown dwarf companion and a distant white dwarf tertiary very much resembles the triple system containing the first transiting planet candidate around a white dwarf ever discovered: WD 1856+534. We have previously argued that the inner binary in WD 1856+534 most likely formed through common envelope evolution but triple star dynamics represent an alternative scenario. Here we analyze different formation scenarios for Gaia 0007-1605. We reconstructed the potential common envelope evolution of the system and find that assuming standard parameters for the energy budget provides a reasonable solution. In agreement with other close white dwarf + brown dwarf binaries, and in contrast to WD 1856+534, no energy sources other than orbital energy during common envelope evolution are required to understand the current configuration of the system. In addition, using analytical prescriptions for triple dynamics, we show that Von Zeipel-Lidov-Kozai oscillations might have trigger tidal migration due to high eccentricity incursions (e \gtrsim 0.997). We conclude that the inner binary in Gaia 0007-1605, as its sibling WD 1856+534, formed either through common envelope evolution, triple dynamics or a combination of both mechanisms. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06666-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06666) | **A High Density of Bright Galaxies at $z\approx10$ in the A2744 region**  |
|| Marco Castellano, et al. -- incl., <mark>Nicha Leethochawalit</mark>, <mark>Lilan Yang</mark> |
|*Appeared on*| *2022-12-14*|
|*Comments*| *Submitted to ApJL, 12 pages, 4 figures*|
|**Abstract**| We report the detection of a high density of redshift $z\approx 10$ galaxies behind the foreground cluster Abell 2744, selected from imaging data obtained recently with NIRCam onboard JWST by three programs -- GLASS-JWST, UNCOVER, and DDT-2756. To ensure robust estimates of the lensing magnification $\mu$, we use an improved version of our model that exploits the first epoch of NIRCam images and newly obtained MUSE spectra, and avoids regions with $\mu>5$ where the uncertainty may be higher. We detect seven bright $z\approx 10$ galaxies with demagnified rest-frame $-22 \lesssim M_{\rm UV}\lesssim -19$ mag, over an area of $\sim37$ sq. arcmin. Taking into account photometric incompleteness and the effects of lensing on luminosity and cosmological volume, we find that the density of $z\approx 10$ galaxies in the field is about $10\times$ ($3\times$) larger than the average at $M_{UV}\approx -21 (-20)$ mag reported so far. The density is even higher when considering only the GLASS-JWST data, which are the depeest and the least affected by magnification and incompleteness. The GLASS-JWST field contains 5 out of 7 galaxies, distributed along an apparent filamentary structure of 2 Mpc in projected length, and includes a close pair of candidates with $M_{\rm UV}< -20$ mag having a projected separation of only 16 kpc. These findings suggest the presence of a $z\approx 10$ overdensity in the field. In addition to providing excellent targets for efficient spectroscopic follow-up observations, our study confirms the high density of bright galaxies observed in early JWST observations, but calls for multiple surveys along independent lines of sight to achieve an unbiased estimate of their average density and a first estimate of their clustering. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06683-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06683) | **UV Luminosity Density Results at z>8 from the First JWST/NIRCam Fields:  Limitations of Early Data Sets and the Need for Spectroscopy**  |
|| Rychard Bouwens, et al. -- incl., <mark>Ivana van Leeuwen</mark> |
|*Appeared on*| *2022-12-14*|
|*Comments*| *25 pages, 10 pages, 15 tables, submitted to MNRAS, figure 7 shows the key results*|
|**Abstract**| We have derived luminosity functions, and set constraints on the UV luminosity and SFR density from z~17 to z~8, using the three most-studied JWST/NIRCam data sets, the SMACS0723, GLASS Parallel, and CEERS fields. We first used our own selections on two independent reductions of these datasets using the latest calibrations. 18 z~8, 12 z~10, 5 z~13, and 1 z~17 candidate galaxies are identified over these fields in our primary reductions, with a similar number of candidates in our secondary reductions. We then use these two reductions, applying a quantitative discriminator, to segregate the full set of z>~8 candidates reported over these fields from the literature, into three different samples, ``robust,'' ``solid,'' and ``possible''. Using all of these samples we then derive UV LF and luminosity density results at $z\geq8$, finding substantial differences. For example, including the full set of ``solid'' and ``possible'' z>~12 candidates from the literature, we find UV luminosity densities which are ~7x and ~20x higher than relying on the ``robust'' candidates alone. These results indicate the evolution of the UV LF and luminosity densities at z>~8 is still extremely uncertain, emphasizing the need for spectroscopy and deeper NIRCam+optical imaging to obtain reliable results. Nonetheless, even with the very conservative ``robust'' approach to selections, both from our own and those of other studies, we find the luminosity density from luminous (M(UV)<-19) galaxies to be ~2x larger than is easily achievable using constant star-formation efficiency models, similar to what other early JWST results have suggested. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06791-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06791) | **X-Shooter Survey of Young Intermediate Mass Stars -- I. Stellar  Characterization and Disc Evolution**  |
|| Daniela P. Iglesias, et al. -- incl., <mark>Lionel Siess</mark> |
|*Appeared on*| *2022-12-14*|
|*Comments*| *19 pages, 11 figures, Accepted to MNRAS*|
|**Abstract**| Intermediate mass stars (IMSs) represent the link between low-mass and high-mass stars and cover a key mass range for giant planet formation. In this paper, we present a spectroscopic survey of 241 young IMS candidates with IR-excess, the most complete unbiased sample to date within 300 pc. We combined VLT/X-Shooter spectra with BVR photometric observations and Gaia DR3 distances to estimate fundamental stellar parameters such as Teff, mass, radius, age, and luminosity. We further selected those stars within the intermediate mass range 1.5 <= Mstar/Msun <= 3.5 and discarded old contaminants. We used 2MASS and WISE photometry to study the IR-excesses of the sample, finding 92 previously unidentified stars with IR-excess. We classified this sample into 'protoplanetary', 'hybrid candidates' and 'debris' discs based on their observed fractional excess at 12microns, finding a new population of 17 hybrid disc candidates. We studied inner disc dispersal timescales for {\lambda} < 10{\mu}m and found very different trends for IMSs and low mass stars (LMSs). IMSs show excesses dropping fast during the first 6 Myrs independently of the wavelength, while LMSs show consistently lower fractions of excess at the shortest wavelengths and increasingly higher fractions for longer wavelengths, with slower dispersal rates. In conclusion, this study demonstrates empirically that IMSs dissipate their inner discs very differently than LMSs, providing a possible explanation for the lack of short period planets around IMSs. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06517-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06517) | **NLTE analysis of the methylidyne radical (CH) molecular lines in  metal-poor stellar atmospheres**  |
|| S. A. Popa, et al. -- incl., <mark>M. Bergemann</mark> |
|*Appeared on*| *2022-12-14*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| An analysis of the CH molecule in non-local thermodynamic equilibrium (NLTE) is performed for the physical conditions of cool stellar atmospheres typical of red giants (log g = 2.0, Teff = 4500 K) and the Sun. The aim of the present work is to explore whether the G-band of the CH molecule, which is commonly used in abundance diagnostics of Carbon-Enhanced Metal-Poor (CEMP) stars, is sensitive to NLTE effects. LTE and NLTE theoretical spectra are computed with the MULTI code. We use one-dimensional (1D) LTE hydrostatic MARCS model atmospheres with parameters representing eleven red giant stars with metallicities ranging from [Fe/H] = -4.0 to [Fe/H] = 0.0 and carbon-to-iron ratios [C/Fe] = 0.0, +0.7, +1.5, and +3.0. The CH molecule model is represented by 1981 energy levels, 18377 radiative bound-bound transitions, and 932 photo-dissociation reactions. The rates due to transitions caused by collisions with free electrons and hydrogen atoms are computed using classical recipes. Our calculations suggest that NLTE effects in the statistical equilibrium of the CH molecule are significant and cannot be neglected for precision spectroscopic analysis of C abundances. The NLTE effects in the G-band increase with decreasing metallicity. We show that the C abundances are always under-estimated if LTE is assumed. The NLTE corrections to C abundance inferred from the CH feature range from +0.04 dex for the Sun to +0.21 dex for a red giant with metallicity [Fe/H] = -4.0. Departures from the LTE assumption in the CH molecule are non-negligible and NLTE effects have to be taken into account in the diagnostic spectroscopy based on the CH lines. We show here that the NLTE effects in the optical CH lines are non-negligible for the Sun and red giant stars, but further calculations are warranted to investigate the effects in other regimes of stellar parameters. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure figures/idl.eps</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.06184-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.06184) | **Quantifying the influence of bars on action-based dynamical modelling of  disc galaxies**  |
|| <mark>Soumavo Ghosh</mark>, Wilma H. Trick, <mark>Gregory M. Green</mark> |
|*Appeared on*| *2022-12-14*|
|*Comments*| *18 pages, 10 figures, 2 tables, comments are welcome*|
|**Abstract**| Action-based dynamical modelling, using stars as dynamical tracers, is an excellent diagnostic to estimate the underlying axisymmetric matter distribution of the Milky Way. However, the Milky Way's bar causes non-axisymmetric resonance features in the stellar disc. Using Roadmapping (an action-based dynamical modelling framework to estimate the gravitational potential and the stellar distribution function), we systematically quantify the robustness of action-based modelling in the presence of a bar. We construct a set of test-particle simulations of barred galaxies (with varying bar properties), and apply Roadmapping to different survey volumes (with varying azimuthal position, size) drawn from these barred models. For realistic bar parameters, the global potential parameters are still recovered to within ${\sim \! 1 \! - \! 20}$ percent. However, with increasing bar strength, the best-fit values of the parameters progressively deviate from their true values. This happens due to a combination of radial heating, radial migration, and resonance overlap phenomena in our bar models. Furthermore, the azimuthal location and the size of the survey volumes play important roles in the successful recovery of the parameters. Survey volumes along the bar major axis produce larger (relative) errors in the best-fit parameter values. In addition, the potential parameters are better recovered for survey volumes with larger spatial coverage. As the Sun is located just ${\sim \! 28 \! - \! 33}$ degrees behind the bar's major axis, an estimate for the bar-induced systematic bias -- as provided by this study -- is therefore crucial for future modelling attempts of the Milky Way. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.06434.md
    + _build/html/tmp_2212.06434/./jwst_mapsNGC628.png
    + _build/html/tmp_2212.06434/./jwst_mapsIC5332.png
    + _build/html/tmp_2212.06434/./jwst_mapsNGC1365.png
    + _build/html/tmp_2212.06434/./jwst_mapsNGC7496.png
    + _build/html/tmp_2212.06434/./ngc0628Jeanslengths_.png
    + _build/html/tmp_2212.06434/./ic5332Jeanslengths_.png
    + _build/html/tmp_2212.06434/./ngc1365Jeanslengths_.png
    + _build/html/tmp_2212.06434/./ngc7496Jeanslengths_.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\degree}{\ensuremath{^\circ}}$
$\newcommand{\kmskpc}{ap{km s^{-1} kpc^{-1}}}$
$\newcommand{\meanv}{\ensuremath{<v>}}$
$\newcommand{\vect}[1]{\boldsymbol{#1}}$
$\newcommand{\}{sol}$
$\newcommand{\}{natexlab}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\degree$}{$\ensuremath${^\circ}}$
$\newcommand{$\kmskpc$}{ap{km s^{-1} kpc^{-1}}}$
$\newcommand{$\meanv$}{$\ensuremath${<v>}}$
$\newcommand{$\vect$}[1]{\boldsymbol{#1}}$
$\newcommand{\}{sol}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# PHANGS--JWST First Results: ISM structure on the turbulent Jeans scale in four disk galaxies observed by JWST and ALMA

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.06434-b31b1b.svg)](https://arxiv.org/abs/2212.06434)<mark>Appeared on: 2022-12-14</mark> - _15 pages, 4 figures. Accepted as part of a PHANGS-JWST ApJL Focus Issue_

</div>
<div id="authors">

Sharon E. Meidt, et al. -- incl., <mark>Eva~Schinnerer</mark>, <mark>Janice C. Lee</mark>, <mark>Jonathan~D.~Henshaw</mark>, <mark>Daizhong Liu</mark>, <mark>Rowan J. Smith</mark>

</div>
<div id="abstract">

**Abstract:** JWST/MIRI imaging of the nearby galaxies IC 5332, NGC 628, NGC 1365 and NGC 7496 from PHANGS reveals a richness of gas structures that in each case form a quasi-regular network of interconnected filaments, shells and voids.  We examine whether this multi-scale network of structure is consistent with the fragmentation of the gas disk through gravitational instability.  We use FilFinder to detect the web of filamentary features in each galaxy and determine their characteristic radial and azimuthal spacings.  These spacings are then compared to estimates of the most Toomre-unstable length (a few kpc), the turbulent Jeans length (a few hundred pc) and the disk scale height (tens of pc) reconstructed using PHANGS-ALMA observations of the molecular gas as a dynamical tracer.  Our analysis of the four galaxies targeted in this work indicates that Jeans-scale structure is pervasive.  Future work will be essential for determining how the structure observed in gas disks impacts not only the rate and location of star formation but also how stellar feedback interacts positively or negatively with the surrounding multi-phase gas reservoir.

</div>

<div id="div_fig1">

<img src="tmp_2212.06434/./jwst_mapsNGC628.png" alt="Fig2.1" width="50%"/><img src="tmp_2212.06434/./jwst_mapsIC5332.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** (Top) JWST/MIRI 11.3 micron images and (bottom) FilFinder masks (see $\S$\ref{sec:FilFinder}) for two PHANGS-JWST targets, NGC 0628 (left) and IC 5332 (right).  The MIRI images are displayed on a asinh stretch.  The physical scale is indicated by a white bar in the left panels, while the red ellipse in the right panels positioned at a galactocentric radius $R$=2 kpc marks the orientation of elliptical annuli in the azimuthal spacings of the filaments identified in these two galaxies.  \ (*fig:maps1*)

</div>
<div id="div_fig2">

<img src="tmp_2212.06434/./jwst_mapsNGC1365.png" alt="Fig3.1" width="50%"/><img src="tmp_2212.06434/./jwst_mapsNGC7496.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** (Top) JWST/MIRI 11.3 micron images and (bottom) FilFinder masks (see $\S$\ref{sec:FilFinder}) for two PHANGS-JWST targets, NGC 1365 (left) and NGC 7496 (right).  The MIRI images are displayed on a asinh stretch. The physical scale is indicated by a white bar in the left panels, while the red lines in the right panels illustrate the orientation of one of the segments along which radial spacings are measured in these two galaxies. (deprojected).\ (*fig:maps2*)

</div>
<div id="div_fig3">

<img src="tmp_2212.06434/./ngc0628Jeanslengths_.png" alt="Fig4.1" width="25%"/><img src="tmp_2212.06434/./ic5332Jeanslengths_.png" alt="Fig4.2" width="25%"/><img src="tmp_2212.06434/./ngc1365Jeanslengths_.png" alt="Fig4.3" width="25%"/><img src="tmp_2212.06434/./ngc7496Jeanslengths_.png" alt="Fig4.4" width="25%"/>

**Figure 4. -** Comparison between the spacings measured for filaments detected by FilFinder (black) and estimates for the Jeans length, Toomre length, most unstable 2D wavelength and gas scale height (gray, blue, red and purple hatched regions) in (clockwise from top left) NGC 628, IC 5332, NGC 7496 and NGC 1365.  In NGC 628 and IC 5332 the average azimuthal filament spacings in 300-pc wide (projected) radial bins are shown. In NGC 1365 and NGC 7496, the average radial filaments spacings in two radial zones $R<$2.5 kpc and $R>2.5$ kpc are shown.  The dark gray bar depicts the Jeans length estimated (where possible) using a determination of the equilibrium gas scale height show in red (see $\S$\ref{sec:jeanstoomre}), while the light gray band shows the range of values implied adopting a fixed 100-pc scale height.  The light blue band traces the range in Toomre lengths estimated for each gas disk.  The purple band shows the value estimated for $\lambda_{2D}$ where 2D structure preferentially forms as long as $Q$ does not exceed unity.  Note that, given the estimated $\lambda_T$, these $\lambda_{2D}$ estimates would all coincide with $Q>1$, disfavoring structure on this scale (see eq. [\ref{eq:toomre}]). Additional hatched light gray and light blue bands for IC 5332 show, respectively, the Jeans length (forward hatch) and Toomre length (backward hatch) when an atomic gas component is included, assuming a constant $h$=100 pc.
  (*fig:spacings*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

132  publications in the last 7 days.
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers